In [ ]:
#|default_exp net.test_graph_mermaid

In [ ]:
#|export
import fbdev

In [ ]:
#|export
from typing import Type, Optional, Union, Any, Tuple, Dict
import asyncio

from fbdev.comp.port import PortType, PortSpec, PortSpecCollection
from fbdev.comp import BaseComponent
from fbdev.graph.graph_spec import GraphSpec

In [ ]:
class FooComponent1(BaseComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp"),
        PortSpec(PortType.OUTPUT, "out1"),
        PortSpec(PortType.OUTPUT, "out2"),
    )
    
class FooComponent2(BaseComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp1"),
        PortSpec(PortType.INPUT, "inp2"),
        PortSpec(PortType.OUTPUT, "out1"),
        PortSpec(PortType.OUTPUT, "out2"),
    )
    
class FooComponent3(BaseComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.INPUT, "inp1"),
        PortSpec(PortType.INPUT, "inp2"),
        PortSpec(PortType.OUTPUT, "out"),
    )

graph = GraphSpec(PortSpecCollection())

graph.add_graph_port(PortSpec(PortType.INPUT, "inp"))
graph.add_graph_port(PortSpec(PortType.OUTPUT, "out"))

node1 = graph.add_node(FooComponent1)
node2 = graph.add_node(FooComponent2)
node3 = graph.add_node(FooComponent3)

graph.ports.input.inp >> node1.ports.input.inp
node1.ports.output.out1 >> node2.ports.input.inp1
node2.ports.input.inp2 << node1.ports.output.out2
node2.ports.output.out1 >> node3.ports.input.inp1
node2.ports.output.out2 >> node3.ports.input.inp2
node3.ports.output.out >> graph.ports.output.out

graph.display_mermaid()

```mermaid
flowchart 
    subgraph FooComponent1["FooComponent1[]"]
        FooComponent1__C__input.inp[inp]
        FooComponent1__C__output.out1[out1]
        FooComponent1__C__output.out2[out2]
        FooComponent1__C__message.started[[started]]
        FooComponent1__C__message.stopped[[stopped]]
    end
    subgraph FooComponent2["FooComponent2[]"]
        FooComponent2__C__input.inp1[inp1]
        FooComponent2__C__input.inp2[inp2]
        FooComponent2__C__output.out1[out1]
        FooComponent2__C__output.out2[out2]
        FooComponent2__C__message.started[[started]]
        FooComponent2__C__message.stopped[[stopped]]
    end
    subgraph FooComponent3["FooComponent3[]"]
        FooComponent3__C__input.inp1[inp1]
        FooComponent3__C__input.inp2[inp2]
        FooComponent3__C__output.out[out]
        FooComponent3__C__message.started[[started]]
        FooComponent3__C__message.stopped[[stopped]]
    end
    GRAPH__C__message.started[[started]]
    GRAPH__C__message.stopped[[stopped]]
    GRAPH__C__input.inp[inp]
    GRAPH__C__output.out[out]
    FooComponent2__C__output.out2 --> FooComponent3__C__input.inp2
    FooComponent2__C__output.out1 --> FooComponent3__C__input.inp1
    FooComponent1__C__output.out1 --> FooComponent2__C__input.inp1
    FooComponent1__C__output.out2 --> FooComponent2__C__input.inp2
    GRAPH__C__input.inp -.-> FooComponent1__C__input.inp
    FooComponent3__C__output.out -.-> GRAPH__C__output.out
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class FooComponent1__C__input.inp,FooComponent2__C__input.inp1,FooComponent2__C__input.inp2,FooComponent3__C__input.inp1,FooComponent3__C__input.inp2,GRAPH__C__input.inp input;
    class FooComponent1__C__output.out1,FooComponent1__C__output.out2,FooComponent2__C__output.out1,FooComponent2__C__output.out2,FooComponent3__C__output.out,GRAPH__C__output.out output;
    class FooComponent1__C__message.started,FooComponent1__C__message.stopped,FooComponent2__C__message.started,FooComponent2__C__message.stopped,FooComponent3__C__message.started,FooComponent3__C__message.stopped,GRAPH__C__message.started,GRAPH__C__message.stopped message;
```

In [ ]:
graph.display_mermaid(hide_unconnected_ports=True)

```mermaid
flowchart 
    subgraph FooComponent1["FooComponent1[]"]
        FooComponent1__C__input.inp[inp]
        FooComponent1__C__output.out1[out1]
        FooComponent1__C__output.out2[out2]
    end
    subgraph FooComponent2["FooComponent2[]"]
        FooComponent2__C__input.inp1[inp1]
        FooComponent2__C__input.inp2[inp2]
        FooComponent2__C__output.out1[out1]
        FooComponent2__C__output.out2[out2]
    end
    subgraph FooComponent3["FooComponent3[]"]
        FooComponent3__C__input.inp1[inp1]
        FooComponent3__C__input.inp2[inp2]
        FooComponent3__C__output.out[out]
    end
    GRAPH__C__message.started[[started]]
    GRAPH__C__message.stopped[[stopped]]
    GRAPH__C__input.inp[inp]
    GRAPH__C__output.out[out]
    FooComponent2__C__output.out2 --> FooComponent3__C__input.inp2
    FooComponent2__C__output.out1 --> FooComponent3__C__input.inp1
    FooComponent1__C__output.out1 --> FooComponent2__C__input.inp1
    FooComponent1__C__output.out2 --> FooComponent2__C__input.inp2
    GRAPH__C__input.inp -.-> FooComponent1__C__input.inp
    FooComponent3__C__output.out -.-> GRAPH__C__output.out
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class FooComponent1__C__input.inp,FooComponent2__C__input.inp1,FooComponent2__C__input.inp2,FooComponent3__C__input.inp1,FooComponent3__C__input.inp2,GRAPH__C__input.inp input;
    class FooComponent1__C__output.out1,FooComponent1__C__output.out2,FooComponent2__C__output.out1,FooComponent2__C__output.out2,FooComponent3__C__output.out,GRAPH__C__output.out output;
    class GRAPH__C__message.started,GRAPH__C__message.stopped message;
```

In [ ]:
top_graph = GraphSpec(PortSpecCollection(
    PortSpec(PortType.INPUT, "inp"),
    PortSpec(PortType.OUTPUT, "out")
))

graph_node = top_graph.add_node(fbdev.graph.GraphComponentFactory.create_component(graph))
top_graph.ports.input.inp >> graph_node.ports.input.inp
top_graph.ports.output.out << graph_node.ports.output.out

top_graph.display_mermaid()

```mermaid
flowchart 
    subgraph GraphComponent["GraphComponent[]"]
        GraphComponent__C__message.started[[started]]
        GraphComponent__C__message.stopped[[stopped]]
        GraphComponent__C__input.inp[inp]
        GraphComponent__C__output.out[out]
        subgraph GraphComponent__PROP__children[" "]
            subgraph GraphComponent__D__FooComponent1["FooComponent1[]"]
                GraphComponent__D__FooComponent1__C__input.inp[inp]
                GraphComponent__D__FooComponent1__C__output.out1[out1]
                GraphComponent__D__FooComponent1__C__output.out2[out2]
                GraphComponent__D__FooComponent1__C__message.started[[started]]
                GraphComponent__D__FooComponent1__C__message.stopped[[stopped]]
            end
            subgraph GraphComponent__D__FooComponent2["FooComponent2[]"]
                GraphComponent__D__FooComponent2__C__input.inp1[inp1]
                GraphComponent__D__FooComponent2__C__input.inp2[inp2]
                GraphComponent__D__FooComponent2__C__output.out1[out1]
                GraphComponent__D__FooComponent2__C__output.out2[out2]
                GraphComponent__D__FooComponent2__C__message.started[[started]]
                GraphComponent__D__FooComponent2__C__message.stopped[[stopped]]
            end
            subgraph GraphComponent__D__FooComponent3["FooComponent3[]"]
                GraphComponent__D__FooComponent3__C__input.inp1[inp1]
                GraphComponent__D__FooComponent3__C__input.inp2[inp2]
                GraphComponent__D__FooComponent3__C__output.out[out]
                GraphComponent__D__FooComponent3__C__message.started[[started]]
                GraphComponent__D__FooComponent3__C__message.stopped[[stopped]]
            end
        end
    end
    GRAPH__C__input.inp[inp]
    GRAPH__C__output.out[out]
    GRAPH__C__message.started[[started]]
    GRAPH__C__message.stopped[[stopped]]
    GraphComponent__D__FooComponent3__C__output.out -.-> GraphComponent__C__output.out
    GraphComponent__D__FooComponent1__C__output.out1 --> GraphComponent__D__FooComponent2__C__input.inp1
    GraphComponent__D__FooComponent1__C__output.out2 --> GraphComponent__D__FooComponent2__C__input.inp2
    GraphComponent__C__output.out -.-> GRAPH__C__output.out
    GraphComponent__D__FooComponent2__C__output.out1 --> GraphComponent__D__FooComponent3__C__input.inp1
    GraphComponent__D__FooComponent2__C__output.out2 --> GraphComponent__D__FooComponent3__C__input.inp2
    GraphComponent__C__input.inp -.-> GraphComponent__D__FooComponent1__C__input.inp
    GRAPH__C__input.inp -.-> GraphComponent__C__input.inp
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class GraphComponent__C__message.started,GraphComponent__C__message.stopped,GraphComponent__D__FooComponent1__C__message.started,GraphComponent__D__FooComponent1__C__message.stopped,GraphComponent__D__FooComponent2__C__message.started,GraphComponent__D__FooComponent2__C__message.stopped,GraphComponent__D__FooComponent3__C__message.started,GraphComponent__D__FooComponent3__C__message.stopped,GRAPH__C__message.started,GRAPH__C__message.stopped message;
    class GraphComponent__C__input.inp,GraphComponent__D__FooComponent1__C__input.inp,GraphComponent__D__FooComponent2__C__input.inp1,GraphComponent__D__FooComponent2__C__input.inp2,GraphComponent__D__FooComponent3__C__input.inp1,GraphComponent__D__FooComponent3__C__input.inp2,GRAPH__C__input.inp input;
    class GraphComponent__C__output.out,GraphComponent__D__FooComponent1__C__output.out1,GraphComponent__D__FooComponent1__C__output.out2,GraphComponent__D__FooComponent2__C__output.out1,GraphComponent__D__FooComponent2__C__output.out2,GraphComponent__D__FooComponent3__C__output.out,GRAPH__C__output.out output;
    class GraphComponent__PROP__children subgraph_zone;
```

In [ ]:
top_graph.display_mermaid(hide_unconnected_ports=True)

```mermaid
flowchart 
    subgraph GraphComponent["GraphComponent[]"]
        GraphComponent__C__input.inp[inp]
        GraphComponent__C__output.out[out]
        subgraph GraphComponent__PROP__children[" "]
            subgraph GraphComponent__D__FooComponent1["FooComponent1[]"]
                GraphComponent__D__FooComponent1__C__input.inp[inp]
                GraphComponent__D__FooComponent1__C__output.out1[out1]
                GraphComponent__D__FooComponent1__C__output.out2[out2]
            end
            subgraph GraphComponent__D__FooComponent2["FooComponent2[]"]
                GraphComponent__D__FooComponent2__C__input.inp1[inp1]
                GraphComponent__D__FooComponent2__C__input.inp2[inp2]
                GraphComponent__D__FooComponent2__C__output.out1[out1]
                GraphComponent__D__FooComponent2__C__output.out2[out2]
            end
            subgraph GraphComponent__D__FooComponent3["FooComponent3[]"]
                GraphComponent__D__FooComponent3__C__input.inp1[inp1]
                GraphComponent__D__FooComponent3__C__input.inp2[inp2]
                GraphComponent__D__FooComponent3__C__output.out[out]
            end
        end
    end
    GRAPH__C__input.inp[inp]
    GRAPH__C__output.out[out]
    GRAPH__C__message.started[[started]]
    GRAPH__C__message.stopped[[stopped]]
    GraphComponent__D__FooComponent3__C__output.out -.-> GraphComponent__C__output.out
    GraphComponent__D__FooComponent1__C__output.out1 --> GraphComponent__D__FooComponent2__C__input.inp1
    GraphComponent__D__FooComponent1__C__output.out2 --> GraphComponent__D__FooComponent2__C__input.inp2
    GraphComponent__C__output.out -.-> GRAPH__C__output.out
    GraphComponent__D__FooComponent2__C__output.out1 --> GraphComponent__D__FooComponent3__C__input.inp1
    GraphComponent__D__FooComponent2__C__output.out2 --> GraphComponent__D__FooComponent3__C__input.inp2
    GraphComponent__C__input.inp -.-> GraphComponent__D__FooComponent1__C__input.inp
    GRAPH__C__input.inp -.-> GraphComponent__C__input.inp
    classDef input fill:#13543e;
    classDef output fill:#0d1b59;
    classDef subgraph_zone fill:#000;
    class GraphComponent__C__input.inp,GraphComponent__D__FooComponent1__C__input.inp,GraphComponent__D__FooComponent2__C__input.inp1,GraphComponent__D__FooComponent2__C__input.inp2,GraphComponent__D__FooComponent3__C__input.inp1,GraphComponent__D__FooComponent3__C__input.inp2,GRAPH__C__input.inp input;
    class GraphComponent__C__output.out,GraphComponent__D__FooComponent1__C__output.out1,GraphComponent__D__FooComponent1__C__output.out2,GraphComponent__D__FooComponent2__C__output.out1,GraphComponent__D__FooComponent2__C__output.out2,GraphComponent__D__FooComponent3__C__output.out,GRAPH__C__output.out output;
    class GraphComponent__PROP__children subgraph_zone;
    class GRAPH__C__message.started,GRAPH__C__message.stopped message;
```